In [ ]:


DEVICE = 'cuda:0' if torch.cuda.is_available() else 'cpu'

model = InceptionResnetV1(
    pretrained="vggface2",
    classify=True,
    num_classes=1,
    device=DEVICE
)


model.to(DEVICE)
model.eval()



criterion = nn.BCEWithLogitsLoss()  # Binary Cross Entropy for single-class classification
optimizer = optim.Adam(model.parameters(), lr=0.001)

import torch
import torch.nn.functional as F
!pip install facenet-pytorch
from facenet_pytorch import MTCNN, InceptionResnetV1
!pip install facenet_pytorch
import numpy as np
from PIL import Image
import cv2
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models

import warnings
warnings.filterwarnings("ignore")
import numpy as np
from PIL import Image
import cv2
import tqdm as tqdm
#l1_strength = 0.001  # Adjust this parameter based on your preference
# Optimizer with L1 Regularization
#optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=l1_strength)

mtcnn = MTCNN(
    select_largest=False,
    post_process=False,
    device=DEVICE
).to(DEVICE).eval()




train_dir = "/kaggle/input/deepfake-and-real-images/Dataset/Train/"

training_data = CustomImageDataset("/kaggle/input/deepfake-and-real-images/Dataset/Validation/Real", "/kaggle/input/deepfake-and-real-images/Dataset/Validation/Fake" , transform)


print(len(training_data))

from torch.utils.data import DataLoader

train_dataloader = DataLoader(training_data, batch_size=64, shuffle=True)


num_epochs = 30

In [ ]:
transform = transforms.Compose([
    transforms.Resize(299),  # InceptionResNetV1 input size
    transforms.CenterCrop(299),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])


class CustomImageDataset(Dataset):
    def __init__(self,  real_dir , fake_dir, transform=None, target_transform=None):
        #self.img_labels = pd.read_csv(annotations_file)
        self.real_dir = real_dir
        self.fake_dir = fake_dir
        self.transform = transform
        self.target_transform = target_transform
        self.real_list = os.listdir(self.real_dir)
        self.fake_list = os.listdir(self.fake_dir)

    def __len__(self):
        return (len(os.listdir(self.real_dir)) + len(os.listdir(self.fake_dir)))

    def __getitem__(self, idx):
        label = 0
        if(idx > len(self.real_list)):
            idx = idx - len(self.real_list)
            file = self.fake_list[idx]
            image = Image.open(os.path.join(self.fake_dir, file))
            label = 0

        else:
           file = self.real_list[idx]
           image = Image.open(os.path.join(self.real_dir, file))
           label = 1
        if self.transform is not None:

            image = self.transform(image)

        return image, label


In [ ]:
from tqdm import tqdm
for epoch in tqdm(range(num_epochs)):
    torch.save(model, 'epoch' + str(epoch) + 'deepfake_detection.pt')
    model.train()
    running_loss = 0.0



    for inputs, labels in train_dataloader:
      model.train()
      inputs = torch.tensor(inputs)
      labels = torch.tensor(labels)
      inputs, labels = inputs.to(DEVICE), labels.to(DEVICE)

      optimizer.zero_grad()
      outputs = model(inputs)
      loss = criterion(outputs.view(-1), labels.float())
      loss.backward()
      optimizer.step()

      running_loss += loss.item()





    epoch_loss = running_loss / len(train_dataloader)
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss:.4f}')